In [ ]:
import os
import tensorflow as tf
from tensorflow import keras as keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import datasets, layers, models, losses, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, Input, BatchNormalization, Dropout
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

Data Preparing

In [ ]:
def convert_data_csv_to_numpy(data, sorting = False): # pandas 통해 읽은 csv data numpy 형태로 변경
  if sorting == True:
    data = data.sort_values(by=[0], axis=0)

  label = np.array(data[0]) # csv file 에서 0번째 colum은 index임
  only_data = np.array(data.drop([0], axis = 1)) # csv file에서 0번 째 colum 탈락 -> data만 남게 됨
  # only_data = tf.convert_to_tensor(only_data, dtype=tf.float32)
  only_data = tf.keras.utils.normalize(only_data, axis=-1, order=2).reshape((-1, 28, 28, 1)) #data nomalize & reshaping
  return only_data, label

In [ ]:
data_dir_path = "../data"

train_df = pd.read_csv(f'{data_dir_path}/emnist-byclass-train.csv', header=None)
test_df = pd.read_csv(f"{data_dir_path}/emnist-byclass-test.csv", header=None)

In [ ]:
X_train, y_train  = convert_data_csv_to_numpy(train_df)
X_test, y_test  = convert_data_csv_to_numpy(test_df)

train_df = None
test_df = None

In [ ]:
# add channel
# channel = 3
# X_train_3 = np.repeat(X_train, channel, axis=-1)
# X_test_3 = np.repeat(X_test, channel, axis=-1)
# y_test_3 = y_test

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
# X_train_3, X_val_3, y_train_3, y_val_3 = train_test_split(X_train_3, y_train, test_size=0.2, random_state=42)

In [ ]:
# data resizing
# X_train = tf.image.resize(X_train, [32,32])
# X_val = tf.image.resize(X_val, [32,32])
# X_test = tf.image.resize(X_test, [32,32])

print(X_train.shape, X_val.shape, X_test.shape)

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train)
y_val = tf.keras.utils.to_categorical(y_val)
y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:
print(y_train.shape,y_val.shape, y_test.shape)

In [ ]:
def show_data_scheme(data, label, row = 4, col = 5, i = 1): # data 어떻게 생겼는지 plot
  label_value_list = []
  for i in range(62):
    if i <=9:
      label_value_list.append(f'{i}')
    elif 10<=i<=35:
      label_value_list.append(f'{chr(i+55)}')
    else:
      label_value_list.append(f'{chr(i+61)}')

  plt.figure(figsize = (11, 8))
  for r in range(row):
    for c in range(col):
      plt.subplot(row, col, i)
      plt.imshow(data[i,...,0], 'gray')
      plt.axis('off')
      plt.title(f'{label_value_list[label[i]]}')
      i+=1
  plt.show()

In [ ]:
show_data_scheme(X_train, y_train)

3. Base line accuracy 측정 

3 - A. Lenet5

In [ ]:
lenet_5_model = keras.models.Sequential([
    keras.layers.Conv2D(6, kernel_size=5, strides=1,  activation='tanh', input_shape=X_train[0].shape, padding='same'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(16, kernel_size=5, strides=1, activation='tanh', padding='valid'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(120, kernel_size=5, strides=1, activation='tanh', padding='valid'),
    keras.layers.Flatten(),   
    keras.layers.Dense(84, activation='tanh'),
    keras.layers.Dense(62, activation='softmax')
])
lenet_5_model.summary()

In [ ]:
lenet_5_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
# model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_lenet5_emnist_model.h5", save_best_only=True)

lenet_5_model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping_cb])

lenet_5_model.evaluate(X_test, y_test)

LeNet_5_SeLU

SeLU performs better(speed & accuracy)

In [ ]:
keras.backend.clear_session()

lenet_5_selu_model = keras.models.Sequential([
    keras.layers.Conv2D(6, kernel_size=5, strides=1,  activation="selu", kernel_initializer="lecun_normal", input_shape=X_train[0].shape, padding='same'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(16, kernel_size=5, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(120, kernel_size=5, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.Flatten(),   
    keras.layers.Dense(84, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(62, activation='softmax')
])

lenet_5_selu_model.summary()

In [ ]:
lenet_5_selu_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
# model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_lenet5_emnist_model.h5", save_best_only=True)

lenet_5_selu_model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), callbacks=[early_stopping_cb])

lenet_5_selu_model.evaluate(X_test, y_test)

3 - B. Mobilenet

In [ ]:
mobile2 = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape = (224, 224, 3), include_top=False)

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(62, activation = 'softmax')

mobile2_model = keras.models.Sequential()
mobile2_model.add(tf.keras.layers.Normalization( axis=-1, mean=44.412914, variance=84.77896, input_shape = (28, 28, 3)))
mobile2_model.add(tf.keras.layers.Resizing(height = 224, width = 224))
mobile2_model.add(mobile2)
mobile2_model.add(global_average_layer)
mobile2_model.add(tf.keras.layers.Dense(1000))
mobile2_model.add(tf.keras.layers.ReLU())
mobile2_model.add(prediction_layer)

mobile2_model.summary()

In [ ]:
mobile2_model.compile(loss="categorical_crossentropy",optimizer="Adam",steps_per_execution=1200,metrics=["accuracy"])

In [ ]:
class LossAndErrorPrintingCallback(keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        print(
            f'''
            epcoh : {epoch}
            loss {logs['loss']}
            accuracy {logs['accuracy']}
            val_loss {logs['val_loss']}
            val_accuracy {logs['val_accuracy']}
            '''
        )


In [ ]:
#  with tf.device("/cpu:0"):
mobile2_model_history = mobile2_model.fit(final_train[:400], train_label[:400], 
                  epochs=10, validation_data=(final_val[:80], val_label[:80]), callbacks=[LossAndErrorPrintingCallback()])

4. Model 선정 및 학습

4 - A.  Lenet base - new architecture

In [ ]:
keras.backend.clear_session()

lenet_5_improved_model = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=5, strides=2, activation="selu", kernel_initializer="lecun_normal", input_shape=X_train[0].shape, padding='same'),
    keras.layers.Conv2D(64, kernel_size=3, strides=2, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.Conv2D(32, kernel_size=1, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(62, activation='softmax')
])

lenet_5_improved_model.summary()

In [ ]:
lenet_5_improved_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
# model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_lenet5_emnist_model.h5", save_best_only=True)

lenet_5_improved_model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping_cb])

lenet_5_improved_model.evaluate(X_test, y_test)

In [ ]:
keras.backend.clear_session()

lenet_5_selu_dropout_model = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=5, strides=1,  activation="selu", kernel_initializer="lecun_normal", input_shape=X_train[0].shape, padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, kernel_size=5, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(32, kernel_size=3, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(62, activation='softmax')
])

lenet_5_selu_dropout_model.summary()

In [ ]:
lenet_5_selu_dropout_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
# model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_lenet5_emnist_model.h5", save_best_only=True)
  
lenet_5_selu_dropout_model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping_cb])

lenet_5_selu_dropout_model.evaluate(X_val, y_val)

In [ ]:
keras.backend.clear_session()

lenet_5_selu_bn_model = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=5, strides=1,  activation="selu", kernel_initializer="lecun_normal", input_shape=X_train[0].shape, padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, kernel_size=3, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(32, kernel_size=3, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(62, activation='softmax')
])

lenet_5_selu_bn_model.summary()

In [ ]:
lenet_5_selu_bn_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
# model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_lenet5_emnist_model.h5", save_best_only=True)
  
lenet_5_selu_bn_model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping_cb])

lenet_5_selu_bn_model.evaluate(X_val, y_val)

4 - B.  Res base - new architecture

In [ ]:
class ResnetBlock(keras.models.Model):
    """
    A standard resnet block.
    """

    def __init__(self, channels: int, down_sample=False):
        """
        channels: same as number of convolution kernels
        """
        super().__init__()

        self.__channels = channels
        self.__down_sample = down_sample
        self.__strides = [2, 1] if down_sample else [1, 1]

        KERNEL_SIZE = (3, 3)
        # use He initialization, instead of Xavier (a.k.a 'glorot_uniform' in Keras), as suggested in [2]
        INIT_SCHEME = "he_normal"

        self.conv_1 = keras.layers.Conv2D(self.__channels, strides=self.__strides[0],
                             kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME)
        self.bn_1 = keras.layers.BatchNormalization()
        self.conv_2 = keras.layers.Conv2D(self.__channels, strides=self.__strides[1],
                             kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME)
        self.bn_2 = keras.layers.BatchNormalization()
        self.merge = keras.layers.Add()

        if self.__down_sample:
            # perform down sampling using stride of 2, according to [1].
            self.res_conv = keras.layers.Conv2D(
                self.__channels, strides=2, kernel_size=(1, 1), kernel_initializer=INIT_SCHEME, padding="same")
            self.res_bn = keras.layers.BatchNormalization()

    def call(self, inputs):
        res = inputs

        x = self.conv_1(inputs)
        x = self.bn_1(x)
        x = tf.nn.relu(x)
        x = self.conv_2(x)
        x = self.bn_2(x)

        if self.__down_sample:
            res = self.res_conv(res)
            res = self.res_bn(res)

        # if not perform down sample, then add a shortcut directly
        x = self.merge([x, res])
        out = tf.nn.relu(x)
        return out

In [ ]:
class ResNet18(keras.models.Model):

    def __init__(self, num_classes, **kwargs):
        """
            num_classes: number of classes in specific classification task.
        """
        super().__init__(**kwargs)
        self.conv_1 = keras.layers.Conv2D(64, (7, 7), strides=2,
                             padding="same", kernel_initializer="he_normal")
        self.init_bn = keras.layers.BatchNormalization()
        self.pool_2 = keras.layers.MaxPool2D(pool_size=(2, 2), strides=2, padding="same")
        self.res_1_1 = ResnetBlock(64)
        self.res_1_2 = ResnetBlock(64)
        self.res_2_1 = ResnetBlock(128, down_sample=True)
        self.res_2_2 = ResnetBlock(128)
        self.res_3_1 = ResnetBlock(256, down_sample=True)
        self.res_3_2 = ResnetBlock(256)
        self.res_4_1 = ResnetBlock(512, down_sample=True)
        self.res_4_2 = ResnetBlock(512)
        self.avg_pool = keras.layers.GlobalAveragePooling2D()
        self.flat = keras.layers.Flatten()
        self.fc = keras.layers.Dense(num_classes, activation="softmax")

    def call(self, inputs):
        out = self.conv_1(inputs)
        out = self.init_bn(out)
        out = tf.nn.relu(out)
        out = self.pool_2(out)
        for res_block in [self.res_1_1, self.res_1_2, self.res_2_1, self.res_2_2, self.res_3_1, self.res_3_2, self.res_4_1, self.res_4_2]:
            out = res_block(out)
        out = self.avg_pool(out)
        out = self.flat(out)
        out = self.fc(out)
        return out

In [ ]:
resnet_model = ResNet18(62)
resnet_model.build(input_shape = (None,28,28,1))
#use categorical_crossentropy since the label is one-hot encoded

# from keras.optimizers import SGD
# opt = SGD(learning_rate=0.1,momentum=0.9,decay = 1e-04) #parameters suggested by He [1]
resnet_model.compile(optimizer = "adam",loss='categorical_crossentropy', metrics=["accuracy"]) 
resnet_model.summary()

Edit input kernel size & seperate upper layers

In [ ]:
class Res_5x5(keras.models.Model):

    def __init__(self, num_classes, **kwargs):
        """
            num_classes: number of classes in specific classification task.
        """
        super().__init__(**kwargs)
        input_ch = 32
        self.conv_1 = keras.layers.Conv2D(input_ch, (5, 5), strides=1,
                             padding="same", kernel_initializer="he_normal")
        self.init_bn = keras.layers.BatchNormalization()
        self.pool_2 = keras.layers.MaxPool2D(pool_size=(2, 2), strides=1, padding="same")
        self.res_1_1 = ResnetBlock(input_ch)
        self.res_1_2 = ResnetBlock(input_ch)
        self.res_2_1 = ResnetBlock(input_ch*2, down_sample=True)
        self.res_2_2 = ResnetBlock(input_ch*2)
        self.res_3_1 = ResnetBlock(input_ch*4, down_sample=True)
        self.res_3_2 = ResnetBlock(input_ch*4)
        
        self.res_num_1 = ResnetBlock(input_ch*8, down_sample=True)
        self.res_num_2 = ResnetBlock(input_ch*8)
        
        self.res_upper_1 = ResnetBlock(input_ch*8, down_sample=True)
        self.res_upper_2 = ResnetBlock(input_ch*8)
        self.res_upper_3 = ResnetBlock(input_ch*8)
   

        self.res_lower_1 = ResnetBlock(input_ch*8, down_sample=True)
        self.res_lower_2 = ResnetBlock(input_ch*8)
        self.res_lower_3 = ResnetBlock(input_ch*8)

        self.num_avg_pool = keras.layers.GlobalAveragePooling2D()
        self.upper_avg_pool = keras.layers.GlobalAveragePooling2D()
        self.lower_avg_pool = keras.layers.GlobalAveragePooling2D()

        self.num_flat = keras.layers.Flatten()
        self.upper_flat = keras.layers.Flatten()
        self.lower_flat = keras.layers.Flatten()

        self.fc_out = keras.layers.Dense(num_classes, activation="softmax")

    def call(self, inputs):
        out = self.conv_1(inputs)
        out = self.init_bn(out)
        out = tf.nn.relu(out)
        out = self.pool_2(out)
        for res_block in [self.res_1_1, self.res_1_2, self.res_2_1, self.res_2_2, self.res_3_1, self.res_3_2]:
            out = res_block(out)
        
        num_out = self.res_num_1(out)
        num_out = self.res_num_2(num_out)
        num_out = self.num_avg_pool(num_out)
        num_out = self.num_flat(num_out)

        upper_out = self.res_upper_1(out)
        upper_out = self.res_upper_2(upper_out)
        upper_out = self.res_upper_3(upper_out)
        upper_out = self.upper_avg_pool(upper_out)
        upper_out = self.upper_flat(upper_out)

        lower_out = self.res_lower_1(out)
        lower_out = self.res_lower_2(lower_out)
        lower_out = self.res_lower_3(lower_out)
        lower_out = self.lower_avg_pool(lower_out)
        lower_out = self.lower_flat(lower_out)

        out = keras.layers.Concatenate()([num_out, upper_out, lower_out])

        out = self.fc_out(out)
        return out



In [ ]:
res_base_model = Res_5x5(62)
res_base_model.build(input_shape = (None,28,28,1))
#use categorical_crossentropy since the label is one-hot encoded

# from keras.optimizers import SGD
# opt = SGD(learning_rate=0.1,momentum=0.9,decay = 1e-04) #parameters suggested by He [1]
res_base_model.compile(optimizer = "adam",loss='categorical_crossentropy', metrics=["accuracy"]) 

In [ ]:
aug = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)

es = EarlyStopping(patience= 5, restore_best_weights=True, monitor="val_accuracy")

batch_size = 256
#I did not use cross validation, so the validate performance is not accurate.
STEPS = len(X_train) / 256

res_base_history = resnet_model.fit(aug.flow(X_train,y_train,batch_size = batch_size), 
      steps_per_epoch=STEPS, 
      batch_size = batch_size, 
      epochs=50, 
      validation_data=(X_val, y_val),
      callbacks=[es])

Res + Dense layer

In [ ]:
class Res_dense(keras.models.Model):

    def __init__(self, **kwargs):
        """
            num_classes: number of classes in specific classification task.
        """
        super().__init__(**kwargs)
        input_ch = 32
        self.conv_1 = keras.layers.Conv2D(input_ch, (5, 5), strides=1,
                             padding="same", kernel_initializer="he_normal")
        self.BN = keras.layers.BatchNormalization()
        self.pool_2 = keras.layers.MaxPool2D(pool_size=(2, 2), strides=1, padding="same")
        self.res_1_1 = ResnetBlock(input_ch)
        self.res_1_2 = ResnetBlock(input_ch)
        self.res_2_1 = ResnetBlock(input_ch*2, down_sample=True)
        self.res_2_2 = ResnetBlock(input_ch*2)
        self.res_2_1 = ResnetBlock(input_ch*4, down_sample=True)
        self.res_2_2 = ResnetBlock(input_ch*4)
        
        self.avg_pool = keras.layers.GlobalAveragePooling2D()
        self.flat = keras.layers.Flatten()
        
        self.res_num_1 = keras.layers.Dense(46, activation="relu", kernel_initializer="he_normal")
        self.res_num_1 = keras.layers.Dense(28, activation="relu", kernel_initializer="he_normal")
        self.res_num_3 = keras.layers.Dense(10, activation = 'softmax')
        
        self.res_upper_1 = keras.layers.Dense(40, activation="relu", kernel_initializer="he_normal")
        self.res_upper_2 = keras.layers.Dense(26, activation = 'softmax')
   

        self.res_lower_1 = keras.layers.Dense(48, activation="relu", kernel_initializer="he_normal")
        self.res_lower_1 = keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal")
        self.res_lower_3 = keras.layers.Dense(26, activation = 'softmax')


    def call(self, inputs):
        out = self.BN(inputs)
        out = self.conv_1(out)
        out = self.BN(out)
        out = self.pool_2(out)
        for res_block in [self.res_1_1, self.res_1_2, self.res_2_1, self.res_2_2]:
            out = res_block(out)
        
        out = self.avg_pool(out)
        out = self.flat(out)

        num_out = self.res_num_1(out)
        num_out = self.BN(num_out)
        num_out = self.res_num_2(num_out)
        num_out = self.BN(num_out)
        num_out = self.res_num_3(num_out)

        upper_out = self.res_upper_1(out)
        upper_out = self.BN(upper_out)
        upper_out = self.res_upper_2(upper_out)

        lower_out = self.res_lower_1(out)
        lower_out = self.BN(lower_out)
        lower_out = self.res_lower_2(lower_out)
        lower_out = self.BN(lower_out)
        lower_out = self.res_lower_3(lower_out)

        out = keras.layers.Concatenate()([num_out, upper_out, lower_out])

        return out



In [ ]:
res_dense_model = Res_dense()
res_dense_model.build(input_shape = (None,28,28,1))
#use categorical_crossentropy since the label is one-hot encoded

# from keras.optimizers import SGD
# opt = SGD(learning_rate=0.1,momentum=0.9,decay = 1e-04) #parameters suggested by He [1]
res_dense_model.compile(optimizer = "adam",loss='categorical_crossentropy', metrics=["accuracy"]) 

In [ ]:
aug = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)

es = EarlyStopping(patience= 5, restore_best_weights=True, monitor="val_accuracy")

batch_size = 256
#I did not use cross validation, so the validate performance is not accurate.
STEPS = len(X_train) / 256

res_dense_history = resnet_model.fit(aug.flow(X_train,y_train,batch_size = batch_size), 
      steps_per_epoch=STEPS, 
      batch_size = batch_size, 
      epochs=50, 
      validation_data=(X_val, y_val),
      callbacks=[es])

4 - C.  VGG – pre-trained

In [ ]:
inputs = Input(shape = (28,28,1))
x = Conv2D (filters =32, kernel_size =3, padding ='same', activation='relu')(inputs)
x = Conv2D (filters =32, kernel_size =3, padding ='same', activation='relu')(x)
x= BatchNormalization()(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 2nd Conv Block
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
x= BatchNormalization()(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 3rd Conv block  
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x) 
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x) 
x= BatchNormalization()(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 4th Conv block
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x= BatchNormalization()(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 5th Conv block
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x= BatchNormalization()(x)
x= Dropout(0.3)(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# Fully connected layers  
x = Flatten()(x)
x= BatchNormalization()(x)
x = Dense(units = 4096, activation ='relu')(x)
x = Dense(units = 4096, activation ='relu')(x)
output = Dense(units = 62, activation ='softmax')(x)
# creating the model

VGG_model = Model (inputs=inputs, outputs =output)
VGG_model.summary()

In [ ]:
VGG_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.005), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
  
VGG_model.fit(X_train, y_train, batch_size = 32, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping_cb])

VGG_model.evaluate(X_test, y_test)

In [ ]:
inputs = Input(shape = (28,28,1))
x = Conv2D (filters =32, kernel_size =3, padding ='same', activation='relu')(inputs)
x = Conv2D (filters =32, kernel_size =3, padding ='same', activation='relu')(x)
x= BatchNormalization()(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 2nd Conv Block
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
x= BatchNormalization()(x)
x= Dropout(0.3)(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 3rd Conv block  
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x) 
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x) 
x= BatchNormalization()(x)
x= Dropout(0.3)(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 4th Conv block
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x= BatchNormalization()(x)
x= Dropout(0.3)(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 5th Conv block
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x= BatchNormalization()(x)
x= Dropout(0.3)(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# Fully connected layers  
x = Flatten()(x)
x= BatchNormalization()(x)
x = Dense(units = 4096, activation ='relu')(x)
x = Dense(units = 4096, activation ='relu')(x)
output = Dense(units = 62, activation ='softmax')(x)
# creating the model

VGG_model_2 = Model (inputs=inputs, outputs =output)
VGG_model_2.summary()

In [ ]:
VGG_model_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.005), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
# model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_new_emnist_model.h5", save_best_only=True)
  
VGG_model_2.fit(X_train, y_train, batch_size = 32, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping_cb])

VGG_model_2.evaluate(X_test, y_test)

In [ ]:
inputs = Input(shape = (28,28,1))
x = Conv2D (filters =32, kernel_size =3, padding ='same', activation='relu')(inputs)
x = Conv2D (filters =32, kernel_size =3, padding ='same', activation='relu')(x)
x= BatchNormalization()(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 2nd Conv Block
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
x= BatchNormalization()(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 3rd Conv block  
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x) 
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x) 
x= BatchNormalization()(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 4th Conv block
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x= BatchNormalization()(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 5th Conv block
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x= BatchNormalization()(x)
x= Dropout(0.3)(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# Fully connected layers  
x = Flatten()(x)
x= BatchNormalization()(x)
x = Dense(units = 4096, activation ='relu')(x)
x = Dense(units = 4096, activation ='relu')(x)
output = Dense(units = 62, activation ='softmax')(x)
# creating the model

VGG_model_3 = Model (inputs=inputs, outputs =output)
VGG_model_3.summary()

In [ ]:
VGG_model_3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.01), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
# model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_new_emnist_model.h5", save_best_only=True)
  
VGG_model_3.fit(X_train, y_train, batch_size = 64, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping_cb])

VGG_model_3.evaluate(X_test, y_test)

In [ ]:
#new VGG
inputs = Input(shape = (28,28,1))
x = Conv2D (filters =32, kernel_size =3, padding ='same', activation='relu')(inputs)
x = Conv2D (filters =32, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 2nd Conv Block
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 3rd Conv block  
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x) 
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x) 
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 4th Conv block
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 5th Conv block
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# Fully connected layers  
x = Flatten()(x)
x = BatchNormalization()(x)
x = Dense(units = 4096, activation ='relu')(x)
x = BatchNormalization()(x)
x = Dense(units = 4096, activation ='relu')(x)
x = BatchNormalization()(x)
output = Dense(units = 62, activation ='softmax')(x)
# creating the model

VGG_model_4 = Model (inputs=inputs, outputs =output)
VGG_model_4.summary()

In [ ]:
VGG_model_4.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.005), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
# model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_new_model.h5", save_best_only=True)
  
VGG_model_4.fit(X_train, y_train, batch_size = 64, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping_cb])

VGG_model_4.evaluate(X_test, y_test)

4 - D.  Efficient net - pre-trained

Fine Tuning

In [ ]:
def plt_learning_curve(history, model_name):
  pd.DataFrame(history.history).plot(figsize=(8, 5))
  plt.grid(True)
  plt.gca().set_ylim(0, 1)
  plt.title(f"{model_name}'s learnig curve", fontsize=14)
  plt.show()

PreProcessing

In [ ]:
def data_crop(data, mean):
  # images = np.zeros((data.shape[0], 20, 20))
  for i in range(data.shape[0]):
    image = data[i]
    first_norm = 255/np.max(image)
    image = (image*first_norm).astype(np.uint8)

    _, binary_image = cv2.threshold(image.astype(np.uint8), 20, 255, cv2.THRESH_BINARY)
    min_max_list = np.where(binary_image==255)
    x_min, x_max = np.min(min_max_list[1]), np.max(min_max_list[1])
    y_min, y_max = np.min(min_max_list[0]), np.max(min_max_list[0])

    if abs(x_max - x_min) >abs(y_max - y_min):
      add_sub_factor = (abs(x_max - x_min) - abs(y_max - y_min))//2
      y_min = max(0, y_min-add_sub_factor-1)
      y_max = min(27, y_max+add_sub_factor+1)

    if abs(y_max - y_min) > abs(x_max - x_min):
      add_sub_factor = (abs(y_max - y_min)-abs(x_max - x_min))//2
      x_min = max(0, x_min-add_sub_factor-1)
      x_max = min(27, x_max+add_sub_factor+1)
    # print(x_min, x_max)
    # print(y_min, y_max)
    data[i] = np.pad(cv2.resize(image.astype(np.uint8)[y_min:y_max, x_min:x_max], (20, 20)), ((4, 4), (4, 4)), 'constant', constant_values = 0)
    second_norm = 255/np.max(data[i])
    data[i] = (data[i]*second_norm).astype(np.uint8)

In [ ]:
tf.keras.utils.normalize(nparr, axis=-1, order=2)